In [ ]:
import pandas as pd
import glob
import shutil
import os
from datetime import datetime

In [2]:
df = pd.read_excel('Sample Data - Vald.xlsx')

In [22]:
paired_positions = [
{
"names": ["Hip Extension - Standing", "Hip Extension - Custom"],
"rules": [
{"condition": lambda hes, hec: -15 <= hes <= 15 and -15 <= hec <= 15, "result": None},
{"condition": lambda hes, hec: hes >= 15 and -15 <= hec <= 15, "result": "LSI_ext + LFA_flex"},
{"condition": lambda hes, hec: hes <= -15 and -15 <= hec <= 15, "result": "RSI_ext + RFA_flex"},
{"condition": lambda hes, hec: -15 <= hes <= 15 and hec >= 15, "result": "LSI_flex + LFA_flex"},
{"condition": lambda hes, hec: -15 <= hes <= 15 and hec <= -15, "result": "RSI_flex + RFA_flex"},
{"condition": lambda hes, hec: hes >= 15 and hec >= 15 and hes > hec, "result": "LFA_flex + LSI_ext"},
{"condition": lambda hes, hec: hes >= 15 and hec >= 15 and hes < hec, "result": "LFA_flex + LSI_flex"},
{"condition": lambda hes, hec: hes <= -15 and hec <= -15 and hes > hec, "result": "RFA_flex + RSI_ext"},
{"condition": lambda hes, hec: hes <= -15 and hec <= -15 and hes < hec, "result": "RFA_flex + RSI_flex"},
{"condition": lambda hes, hec: hes >= 15 and hec <= -15, "result": "LSI_ext + LFA_flex and RSI_flex + RFA_flex"},
{"condition": lambda hes, hec: hes <= -15 and hec >= 15, "result": "RSI_ext + RFA_flex and LSI_flex + LFA_flex"},
],
},
{
"names": ["Hip Flexion - Supine", "Hip Flexion - Seated"],
"rules": [
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and -15 <= hfse <= 15, "result": None},
{"condition": lambda hfsu, hfse: hfsu >= 15 and -15 <= hfse <= 15, "result": "RSI_ext + RFA_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and -15 <= hfse <= 15, "result": "LSI_ext + LFA_ext"},
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and hfse >= 15, "result": "RSI_flex + RFA_ext"},
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and hfse <= -15, "result": "LSI_flex + LFA_ext"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse >= 15 and hfsu > hfse, "result": "RFA_ext + RSI_ext"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse >= 15 and hfsu < hfse, "result": "RFA_ext + RSI_flex"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse <= -15 and hfsu > hfse, "result": "LFA_ext + LSI_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse <= -15 and hfsu < hfse, "result": "LFA_ext + LSI_flex"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse <= -15, "result": "RSI_ext + RFA_ext and LSI_flex + LFA_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse >= 15, "result": "LSI_ext + LFA_ext and RSI_flex + RFA_ext"},
],
},
# Add more pairs as needed
]


unpaired_tests = [
{
"name": "Knee Flexion - Prone",
"rules": [
{"condition": lambda kfp: -15 <= kfp <= 15, "result": None},
{"condition": lambda kfp: kfp >= 15, "result": "LK_ext"},
{"condition": lambda kfp: kfp <= -15, "result": "RK_ext"},
],
},
{
"name": "Knee Extension",
"rules": [
{"condition": lambda ke: -15 <= ke <= 15, "result": None},
{"condition": lambda ke: ke >= 15, "result": "LK_flex"},
{"condition": lambda ke: ke <= -15, "result": "RK_flex"},
],
},
{
"name": "Ankle Dorsiflexion - Seated",
"rules": [
{"condition": lambda ads: -15 <= ads <= 15, "result": None},
{"condition": lambda ads: ads >= 15, "result": "LA_plantarflexion"},
{"condition": lambda ads: ads <= -15, "result": "RA_plantarflexion"},
],
},
{
"name": "Ankle Plantar flexion",
"rules": [
{"condition": lambda apf: -15 <= apf <= 15, "result": None},
{"condition": lambda apf: apf >= 15, "result": "LA_dorsiflexion"},
{"condition": lambda apf: apf <= -15, "result": "RA_dorsiflexion"},
],
},
]
complex_tests = [
{
"names": ["Hip AD/AB - 90 - Squeeze", "Hip AD/AB - Seated - Squeeze"],
"rules": [
{"condition": lambda had90, hads: -15 <= had90 <= 15 and -15 <= hads <= 15, "result": None},
{"condition": lambda had90, hads: had90 >= 15 and -15 <= hads <= 15, "result": "LSI_ext + LFA_flexion/abduction"},
{"condition": lambda had90, hads: -15 <= had90 <= 15 and hads >= 15, "result": "LSI_flex + LFA_flexion/abduction"},
{"condition": lambda had90, hads: had90 >= 15 and hads >= 15 and hads < had90, "result": "LFA_flexion/abduction + LSI_ext"},
{"condition": lambda had90, hads: had90 >= 15 and hads >= 15 and hads > had90, "result": "LFA_flexion/abduction + LSI_flex"},
{"condition": lambda had90, hads: had90 <= -15 and hads <= -15 and hads < had90, "result": "RFA_flexion/abduction + RSI_ext"},
{"condition": lambda had90, hads: had90 <= -15 and hads <= -15 and hads > had90, "result": "RFA_flexion/abduction + RSI_flex"},
{"condition": lambda had90, hads: had90 <= -15 and -15 <= hads <= 15, "result": "RSI_ext + RFA_flexion/abduction"},
{"condition": lambda had90, hads: -15 <= had90 <= 15 and hads <= -15, "result": "RSI_flex + RFA_flexion/abduction"},
],
},
{
"names": ["Hip AD/AB - 90 - Pull", "Hip AD/AB - Seated - Pull"],
"rules": [
{"condition": lambda hab90, habs: -15 <= hab90 <= 15 and -15 <= habs <= 15, "result": None},
{"condition": lambda hab90, habs: hab90 >= 15 and -15 <= habs <= 15, "result": "LSI_ext + LFA_flexion/adduction"},
{"condition": lambda hab90, habs: -15 <= hab90 <= 15 and habs >= 15, "result": "LSI_flex + LFA_flexion/adduction"},
{"condition": lambda hab90, habs: hab90 >= 15 and habs >= 15 and habs < hab90, "result": "LFA_flexion/adduction + LSI_ext"},
{"condition": lambda hab90, habs: hab90 >= 15 and habs >= 15 and habs > hab90, "result": "LFA_flexion/adduction + LSI_flex"},
{"condition": lambda hab90, habs: hab90 <= -15 and habs <= -15 and habs < hab90, "result": "RFA_flexion/adduction + RSI_ext"},
{"condition": lambda hab90, habs: hab90 <= -15 and habs <= -15 and habs > hab90, "result": "RFA_flexion/adduction + RSI_flex"},
{"condition": lambda hab90, habs: hab90 <= -15 and -15 <= habs <= 15, "result": "RSI_ext + RFA_flexion/adduction"},
{"condition": lambda hab90, habs: -15 <= hab90 <= 15 and habs <= -15, "result": "RSI_flex + RFA_flexion/adduction"},
],
},
{
"names": ["Hip AD/AB - Supine (Knee) - Squeeze", "Hip AD/AB - Supine (Ankle) - Squeeze"],
"rules": [
{"condition": lambda hadk, hada: -15 <= hadk <= 15 and -15 <= hada <= 15, "result": None},
{"condition": lambda hadk, hada: hadk >= 15 and -15 <= hada <= 15, "result": "LSI_ext + LFA_extension/abduction"},
{"condition": lambda hadk, hada: -15 <= hadk <= 15 and hada >= 15, "result": "LSI_flex + LFA_extension/abduction"},
{"condition": lambda hadk, hada: hadk >= 15 and hada >= 15 and hadk < hada, "result": "LFA_extension/abduction + LSI_ext"},
{"condition": lambda hadk, hada: hadk >= 15 and hada >= 15 and hadk > hada, "result": "LFA_extension/abduction + LSI_flex"},
{"condition": lambda hadk, hada: hadk <= -15 and hada <= -15 and hadk < hada, "result": "RFA_extension/abduction + RSI_ext"},
{"condition": lambda hadk, hada: hadk <= -15 and hada <= -15 and hadk > hada, "result": "RFA_extension/abduction + RSI_flex"},
{"condition": lambda hadk, hada: hadk <= -15 and -15 <= hada <= 15, "result": "RSI_ext + RFA_extension/abduction"},
{"condition": lambda hadk, hada: -15 <= hadk <= 15 and hada <= -15, "result": "RSI_flex + RFA_extension/abduction"},
],
},
{
"names": ["Hip AD/AB - Supine (Knee) - Pull", "Hip AD/AB - Supine (Ankle) - Pull"],
"rules": [
{"condition": lambda habk, haba: -15 <= habk <= 15 and -15 <= haba <= 15, "result": None},
{"condition": lambda habk, haba: habk >= 15 and -15 <= haba <= 15, "result": "LSI_ext + LFA_extension/adduction"},
{"condition": lambda habk, haba: -15 <= habk <= 15 and haba >= 15, "result": "LSI_flex + LFA_extension/adduction"},
{"condition": lambda habk, haba: habk >= 15 and haba >= 15 and habk < haba, "result": "LFA_extension/adduction + LSI_ext"},
{"condition": lambda habk, haba: habk >= 15 and haba >= 15 and habk > haba, "result": "LFA_extension/adduction + LSI_flex"},
{"condition": lambda habk, haba: habk <= -15 and haba <= -15 and habk < haba, "result": "RFA_extension/adduction + RSI_ext"},
{"condition": lambda habk, haba: habk <= -15 and haba <= -15 and habk > haba, "result": "RFA_extension/adduction + RSI_flex"},
{"condition": lambda habk, haba: habk <= -15 and -15 <= haba <= 15, "result": "RSI_ext + RFA_extension/adduction"},
{"condition": lambda habk, haba: -15 <= habk <= 15 and haba <= -15, "result": "RSI_flex + RFA_extension/adduction"},
],
},
{
"names": ["Hip IR/ER - Custom - Pull", "Shoulder IR/ER - Supine (Neutral) - Squeeze"],
"rules": [
{"condition": lambda hirc, hirs: -15 <= hirc <= 15 and -15 <= hirs <= 15, "result": None},
{"condition": lambda hirc, hirs: hirc >= 15 and -15 <= hirs <= 15, "result": "LSI_ext + LFA_flexion/ER"},
{"condition": lambda hirc, hirs: -15 <= hirc <= 15 and hirs >= 15, "result": "LSI_flex + LFA_flexion/ER"},
{"condition": lambda hirc, hirs: hirc >= 15 and hirs >= 15 and hirs < hirc, "result": "LFA_flexion/ER + LSI_ext"},
{"condition": lambda hirc, hirs: hirc >= 15 and hirs >= 15 and hirs > hirc, "result": "LFA_flexion/ER + LSI_flex"},
{"condition": lambda hirc, hirs: hirc <= -15 and hirs <= -15 and hirs < hirc, "result": "RFA_flexion/ER + RSI_ext"},
{"condition": lambda hirc, hirs: hirc <= -15 and hirs <= -15 and hirs > hirc, "result": "RFA_flexion/ER + RSI_flex"},
{"condition": lambda hirc, hirs: hirc <= -15 and -15 <= hirs <= 15, "result": "RSI_ext + RFA_flexion/ER"},
{"condition": lambda hirc, hirs: -15 <= hirc <= 15 and hirs <= -15, "result": "RSI_flex + RFA_flexion/ER"},
],
},
{
"names": ["Hip IR/ER - Custom - Squeeze", "Shoulder IR/ER - Supine (Neutral) - Pull"],
"rules": [
{"condition": lambda herc, hers: -15 <= herc <= 15 and -15 <= hers <= 15, "result": None},
{"condition": lambda herc, hers: herc >= 15 and -15 <= hers <= 15, "result": "LSI_ext + LFA_flexion/IR"},
{"condition": lambda herc, hers: -15 <= herc <= 15 and hers >= 15, "result": "LSI_flex + LFA_flexion/IR"},
{"condition": lambda herc, hers: herc >= 15 and hers >= 15 and herc < hers, "result": "LFA_flexion/IR + LSI_ext"},
{"condition": lambda herc, hers: herc >= 15 and hers >= 15 and herc > hers, "result": "LFA_flexion/IR + LSI_flex"},
{"condition": lambda herc, hers: herc <= -15 and hers <= -15 and herc < hers, "result": "RFA_flexion/IR + RSI_ext"},
{"condition": lambda herc, hers: herc <= -15 and hers <= -15 and herc > hers, "result": "RFA_flexion/IR + RSI_flex"},
{"condition": lambda herc, hers: herc <= -15 and -15 <= hers <= 15, "result": "RSI_ext + RFA_flexion/IR"},
{"condition": lambda herc, hers: -15 <= herc <= 15 and hers <= -15, "result": "RSI_flex + RFA_flexion/IR"},
],
},
{
"names": ["Hip IR/ER - Prone - Pull", "Shoulder IR/ER - Custom - Pull"],
"rules": [
{"condition": lambda hirp, hirsc: -15 <= hirp <= 15 and -15 <= hirsc <= 15, "result": None},
{"condition": lambda hirp, hirsc: hirp >= 15 and -15 <= hirsc <= 15, "result": "LSI_ext + LFA_extension/ER"},
{"condition": lambda hirp, hirsc: -15 <= hirp <= 15 and hirsc >= 15, "result": "LSI_flex + LFA_extension/ER"},
{"condition": lambda hirp, hirsc: hirp >= 15 and hirsc >= 15 and hirp < hirsc, "result": "LFA_extension/ER + LSI_flex"},
{"condition": lambda hirp, hirsc: hirp >= 15 and hirsc >= 15 and hirp > hirsc, "result": "LFA_extension/ER + LSI_ext"},
{"condition": lambda hirp, hirsc: hirp <= -15 and hirsc <= -15 and hirp < hirsc, "result": "RFA_extension/ER + RSI_ext"},
{"condition": lambda hirp, hirsc: hirp <= -15 and hirsc <= -15 and hirp > hirsc, "result": "RFA_extension/ER + RSI_flex"},
{"condition": lambda hirp, hirsc: hirp <= -15 and -15 <= hirsc <= 15, "result": "RSI_ext + RFA_extension/ER"},
{"condition": lambda hirp, hirsc: -15 <= hirp <= 15 and hirsc <= -15, "result": "RSI_flex + RFA_extension/ER"},
],
},
{
"names": ["Hip IR/ER - Prone - Squeeze", "Shoulder IR/ER - Custom Squeeze"],
"rules": [
{"condition": lambda herp, hersc: -15 <= herp <= 15 and -15 <= hersc <= 15, "result": None},
{"condition": lambda herp, hersc: herp >= 15 and -15 <= hersc <= 15, "result": "LSI_ext + LFA_Extension/IR"},
{"condition": lambda herp, hersc: -15 <= herp <= 15 and hersc >= 15, "result": "LSI_flex + LFA_Extension/IR"},
{"condition": lambda herp, hersc: herp >= 15 and hersc >= 15 and herp < hersc, "result": "LFA_Extension/IR + LSI_flex"},
{"condition": lambda herp, hersc: herp >= 15 and hersc >= 15 and herp > hersc, "result": "LFA_Extension/IR + LSI_ext"},
{"condition": lambda herp, hersc: herp <= -15 and hersc <= -15 and herp < hersc, "result": "RFA_Extension/IR + RSI_ext"},
{"condition": lambda herp, hersc: herp <= -15 and hersc <= -15 and herp > hersc, "result": "RFA_Extension/IR + RSI_flex"},
{"condition": lambda herp, hersc: herp <= -15 and -15 <= hersc <= 15, "result": "RSI_ext + RFA_Extension/IR"},
{"condition": lambda herp, hersc: -15 <= herp <= 15 and hersc <= -15, "result": "RSI_flex + RFA_Extension/IR"},
],
},
{
"names": ["Ankle Plantar Flexion IN/EV - Squeeze","Ankle Plantar Flexion IN/EV - Pull"],
"rules": [
{"condition": lambda pfi, pfe: -15 <= pfi <= 15 and -15 <= pfe <= 15, "result": None},
{"condition": lambda pfi, pfe: pfi >= 15 and -15 <= pfe <= 15, "result": "LAE_dorsiflexion"},
{"condition": lambda pfi, pfe: pfi <= -15 and -15 <= pfe <= 15, "result": "RAE_dorsiflexion"},
{"condition": lambda pfi, pfe: -15 <= pfi <= 15 and pfe >= 15, "result": "LAI_dorsiflexion"},
{"condition": lambda pfi, pfe: -15 <= pfi <= 15 and pfe <= -15, "result": "RAI_dorsiflexion"},
{"condition": lambda pfi, pfe: pfi >= 15 and pfe >= 15, "result": "LAE_dorsiflexion and LAI_dorsiflexion"},
{"condition": lambda pfi, pfe: pfi <= -15 and pfe <= -15, "result": "RAE_dorsiflexion and RAI_dorsiflexion"},
{"condition": lambda pfi, pfe: pfi >= 15 and pfe <= -15, "result": "LAE_dorsiflexion and RAI_dorsiflexion"},
{"condition": lambda pfi, pfe: pfi <= -15 and pfe >= 15, "result": "RAE_dorsiflexion and LAI_dorsiflexion"},
],
},
{
"names": ["Ankle Inversion/Eversion - Supine - Squeeze", "Ankle Inversion/Eversion - Supine - Pull"],
"rules": [
{"condition": lambda afi, afe: -15 <= afi <= 15 and -15 <= afe <= 15, "result": None},
{"condition": lambda afi, afe: afi >= 15 and -15 <= afe <= 15, "result": "LAE_plantarflexion"},
{"condition": lambda afi, afe: afi <= -15 and -15 <= afe <= 15, "result": "RAE_plantarflexion"},
{"condition": lambda afi, afe: -15 <= afi <= 15 and afe >= 15, "result": "LAI_plantarflexion"},
{"condition": lambda afi, afe: -15 <= afi <= 15 and afe <= -15, "result": "RAI_plantarflexion"},
{"condition": lambda afi, afe: afi >= 15 and afe >= 15, "result": "LAE_plantarflexion and LAI_plantarflexion"},
{"condition": lambda afi, afe: afi <= -15 and afe <= -15, "result": "RAE_plantarflexion and RAI_plantarflexion"},
{"condition": lambda afi, afe: afi >= 15 and afe <= -15, "result": "LAE_plantarflexion and RAI_plantarflexion"},
{"condition": lambda afi, afe: afi <= -15 and afe >= 15, "result": "RAE_plantarflexion and LAI_plantarflexion"},
],
},
]

# Below this cell is where solution begins

- Start with reformatting your data

In [54]:
better_formatted_paired_positions = [
{ ("Hip Extension - Standing", "Hip Extension - Custom"): [
{"condition": lambda hes, hec: -15 <= hes <= 15 and -15 <= hec <= 15, "result": None},
{"condition": lambda hes, hec: hes >= 15 and -15 <= hec <= 15, "result": "LSI_ext + LFA_flex"},
{"condition": lambda hes, hec: hes <= -15 and -15 <= hec <= 15, "result": "RSI_ext + RFA_flex"},
{"condition": lambda hes, hec: -15 <= hes <= 15 and hec >= 15, "result": "LSI_flex + LFA_flex"},
{"condition": lambda hes, hec: -15 <= hes <= 15 and hec <= -15, "result": "RSI_flex + RFA_flex"},
{"condition": lambda hes, hec: hes >= 15 and hec >= 15 and hes > hec, "result": "LFA_flex + LSI_ext"},
{"condition": lambda hes, hec: hes >= 15 and hec >= 15 and hes < hec, "result": "LFA_flex + LSI_flex"},
{"condition": lambda hes, hec: hes <= -15 and hec <= -15 and hes > hec, "result": "RFA_flex + RSI_ext"},
{"condition": lambda hes, hec: hes <= -15 and hec <= -15 and hes < hec, "result": "RFA_flex + RSI_flex"},
{"condition": lambda hes, hec: hes >= 15 and hec <= -15, "result": "LSI_ext + LFA_flex and RSI_flex + RFA_flex"},
{"condition": lambda hes, hec: hes <= -15 and hec >= 15, "result": "RSI_ext + RFA_flex and LSI_flex + LFA_flex"},
],
},
{ ("Hip Flexion - Supine", "Hip Flexion - Seated"): [
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and -15 <= hfse <= 15, "result": None},
{"condition": lambda hfsu, hfse: hfsu >= 15 and -15 <= hfse <= 15, "result": "RSI_ext + RFA_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and -15 <= hfse <= 15, "result": "LSI_ext + LFA_ext"},
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and hfse >= 15, "result": "RSI_flex + RFA_ext"},
{"condition": lambda hfsu, hfse: -15 <= hfsu <= 15 and hfse <= -15, "result": "LSI_flex + LFA_ext"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse >= 15 and hfsu > hfse, "result": "RFA_ext + RSI_ext"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse >= 15 and hfsu < hfse, "result": "RFA_ext + RSI_flex"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse <= -15 and hfsu > hfse, "result": "LFA_ext + LSI_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse <= -15 and hfsu < hfse, "result": "LFA_ext + LSI_flex"},
{"condition": lambda hfsu, hfse: hfsu >= 15 and hfse <= -15, "result": "RSI_ext + RFA_ext and LSI_flex + LFA_ext"},
{"condition": lambda hfsu, hfse: hfsu <= -15 and hfse >= 15, "result": "LSI_ext + LFA_ext and RSI_flex + RFA_ext"},
],
},
]

In [116]:
def apply_pairwise_conditions(df, movement_pair, lambda_conditions):
    """
    Apply pairwise conditions to a DataFrame based on movement pairs and lambda conditions.

    *** This function can be modified for the unpaired tests to accept only a single parameter for lambda functions ***

    Parameters:
    - df (DataFrame): The DataFrame to process, typically 'temp_df'.
    - movement_pair (tuple): A pair of reciprocal motions as defined in the dictionary of conditions. 
                             Example: ("Hip Extension - Standing", "Hip Extension - Custom").
    - lambda_conditions (list of dicts): The list of dictionaries containing lambda functions and their corresponding results.

    Process:
    1) Iterates over each unique date in the DataFrame. This is based on the assumption that the tests being compared 
       by the lambda functions occur on the same date.
    
    2) For each date, iterates over each subject, under the assumption that each subject performs the tests only once per day.
       IMPORTANT: This is a key assumption of the function. If a subject may perform the same tests multiple times in one day,
       the function will need refactoring to consider the time of day and additional logic to group tests appropriately based on temporal proximity.
       This aspect has not been thoroughly checked.

    3) For each subject on a given date, checks if the specified movements are present. 

    4) Iterates over the lambda conditions, applying them to the 'Max Imbalance' values from the paired movements.

    5) When a matching condition is found, updates the 'Result' column in the DataFrame and breaks out of the loop for efficiency.

    Notes:
    - This function updates the 'Result' column in the original DataFrame directly.
    - Ensure the DataFrame structure aligns with the assumptions of the function for accurate results.
    """

    for date in df['Date UTC'].unique():
        # Filter rows by date so we address it date by date
        df_date = df[df['Date UTC'] == date]

        # Filter the rows by subject because multiple subjects can take the same tests on the same day
        for subject in df_date['Name'].unique():
            df_subject = df_date[df_date['Name']==subject]

            # Filter rows by the specific movements so we can compare the movement pairs which are our new keys
            df_movement_1 = df_subject[df_subject['Position'] == movement_pair[0]]
            df_movement_2 = df_subject[df_subject['Position'] == movement_pair[1]]

            # Ensure both movements are present
            if not df_movement_1.empty and not df_movement_2.empty:

                # Extract 'Max Imbalance' values
                max_imbalance_1 = df_movement_1['Max Imbalance'].iloc[0]
                max_imbalance_2 = df_movement_2['Max Imbalance'].iloc[0]

                # Apply lambda conditions
                for condition in lambda_conditions:
                    if condition['condition'](max_imbalance_1, max_imbalance_2):
                        # Update 'Result' for these rows
                        df.loc[df_date.index, 'Result'] = condition['result']
                        break



In [122]:
# results_df = pd.DataFrame()
paired_names = [("Hip Extension - Standing", "Hip Extension - Custom"), ("Hip Flexion - Supine", "Hip Flexion - Seated")]
filepath = 'Sample Data - Vald.xlsx'

# You need to reformat your unpaired_tests to follow the formatting the new paired_positions follows
# In order for this to work
#  See "better_formatted_paired_positions" vs "paired_positions"

test_group = better_formatted_paired_positions


def process_movement_data(filepath, paired_names, test_group):
    """
    Processes the movement data from an Excel file based on specified conditions.

    Parameters:
    - filepath (str): The path to the Excel file containing the data.
    - paired_names (list of tuples): Pairs of reciprocal motions to be analyzed.
    - test_group (dict): A dictionary of conditions for each pair of movements.

    Returns:
    - DataFrame: The processed DataFrame with results of the lambda conditions applied.

    Process:
    1) Reads the data from the Excel file.
    2) Concatenates data for the specified movement names.
    3) Filters out rows based on 'L Reps' and 'R Reps'.
    4) Applies pairwise conditions using the apply_pairwise_conditions function.
    5) Sorts the DataFrame by index before returning.
    """

    df = pd.read_excel(filepath)

    # List of all movement names
    df_names = [name for pair in paired_names for name in pair]

    # Concatenate DataFrames for each specified position
    pair_dfs = [df[df['Position'] == name] for name in df_names]
    pair_df = pd.concat(pair_dfs)
    temp_df = pair_df.copy()

    # Filter out rows where 'L Reps' and 'R Reps' are less than 2
    temp_df = temp_df[(temp_df['L Reps'] >= 2) & (temp_df['R Reps'] >= 2)]
    temp_df['Result'] = None

    # Apply conditions for each pair of movements
    for dict_, key in zip(test_group, paired_names):
        conditions = dict_[key]
        apply_pairwise_conditions(temp_df, key, conditions)

    # Sort DataFrame by index
    temp_df = temp_df.sort_index()

    return temp_df

results_df = process_movement_data(filepath, paired_names, test_group)
results_df



,Name,ExternalId,Date UTC,Time UTC,Device,Mode,Test,Direction,Position,L Reps,...,L Avg Force (N),R Avg Force (N),Avg Imbalance,L Avg Ratio,R Avg Ratio,L Max Impulse (Ns),R Max Impulse (Ns),Impulse Imbalance (%),Notes,Result
10,Subject 1,NaN,2023-11-10,15:24:00,ForceFrame-2086,Bar + Frame,Hip Extension,Pull,Hip Extension - Custom,3,...,299.083333,412.250000,27.450980,NaN,NaN,3511.130,3897.595,9.915474,NaN,None
12,Subject 1,NaN,2023-11-10,15:20:00,ForceFrame-2086,Bar + Frame,Hip Extension,Pull,Hip Extension - Standing,3,...,276.583333,349.416667,20.844264,NaN,0.00,2657.500,4809.195,44.741272,NaN,None
16,Subject 1,NaN,2023-11-10,15:10:00,ForceFrame-2086,Bar + Frame,Hip Flexion,Pull,Hip Flexion - Seated,3,...,204.583333,182.833333,-10.631365,0.08,0.01,1980.035,2828.485,29.996624,NaN,None
18,Subject 1,NaN,2023-11-10,15:07:00,ForceFrame-2086,Bar + Frame,Hip Flexion,Pull,Hip Flexion - Supine,3,...,324.500000,275.583333,-15.074474,NaN,NaN,4313.500,2850.235,-33.922916,NaN,None
26,Subject 1,NaN,2023-08-04,15:40:00,ForceFrame-1997,Bar + Frame,Hip Extension,Pull,Hip Extension - Custom,3,...,343.958333,380.916667,9.702472,NaN,NaN,5375.205,5141.450,-4.348764,NaN,LFA_ext + LSI_ext
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,Subject 9,NaN,2022-10-17,22:18:00,ForceFrame-1371,Bar + Frame,Hip Flexion,Pull,Hip Flexion - Supine,3,...,266.333333,315.250000,15.516786,NaN,NaN,848.510,1062.570,20.145496,NaN,RSI_ext + RFA_ext
1280,Subject 9,NaN,2022-10-07,21:59:00,ForceFrame-1371,Bar + Frame,Hip Extension,Pull,Hip Extension - Custom,3,...,216.083333,205.000000,-5.129194,0.03,0.00,716.480,687.765,-4.007788,NaN,LFA_ext + LSI_ext
1282,Subject 9,NaN,2022-10-07,21:57:00,ForceFrame-1371,Bar + Frame,Hip Extension,Pull,Hip Extension - Standing,3,...,179.083333,175.583333,-1.954397,NaN,0.01,954.140,607.535,-36.326430,NaN,LFA_ext + LSI_ext
1286,Subject 9,NaN,2022-10-07,21:47:00,ForceFrame-1371,Bar + Frame,Hip Flexion,Pull,Hip Flexion - Seated,3,...,229.250000,180.833333,-21.119593,NaN,NaN,610.265,459.225,-24.749904,NaN,LFA_ext + LSI_ext


In [ ]:

# for test in complex_tests:
# # Concatenate Test and Direction to form unique test names
#     test_names = test["names"]
#     test_dfs = [df[(df['Position'] + ' - ' + df['Direction'] == test_name)] for test_name in test_names]
#     if all(len(test_df) > 0 for test_df in test_dfs):
#         test_df = pd.concat(test_dfs)
# for date in test_df['Date UTC'].unique():
#     temp_df = test_df[test_df['Date UTC'] == date]
# # Filter out rows where 'L Reps' and 'R Reps' are less than 2
#     temp_df = temp_df[(temp_df['L Reps'] >= 2) & (temp_df['R Reps'] >= 2)]
# # Initialize Result column
#     temp_df['Result'] = None
# for rule in test["rules"]:
#     if rule["condition"](*temp_df['Max Imbalance'].values):
#         temp_df['Result'] = rule["result"]
#         results_df = pd.concat([results_df, temp_df])



# for test in unpaired_tests:
#     test_df = df[df['Position'] == test["name"]]
#     if len(test_df) > 0:
# # Filter out rows where 'L Reps' and 'R Reps' are less than 2
#         test_df = test_df[(test_df['L Reps'] >= 2) & (test_df['R Reps'] >= 2)]
# # Initialize Result column
#         test_df['Result'] = None
# for index, row in test_df.iterrows():
#     for rule in test["rules"]:
#         if rule["condition"](row['Max Imbalance']):
#             test_df.at[index, 'Result'] = rule["result"]
#             results_df = pd.concat([results_df, test_df])
# # Select only the desired columns
#         results_df = results_df[['Name', 'Date UTC', 'Direction', 'Position', 'L Reps', 'R Reps', 'L Max Force (N)', 'R Max Force (N)', 'Max Imbalance', 'Result']]

# # Get the current date and time
# now = datetime.now()

# # Format it as a string in the format Year-Month-Day_Hour-Minute-Second
# timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')

# # Create the filename by using just the timestamp and the file extension
# output_file_name = f'{timestamp}.csv'

# # Make sure the "Results" directory exists, and if not, create it
# if not os.path.exists('Results'):
#     os.makedirs('Results')

# # Save the results dataframe to a csv file in the "Results" directory
#     results_df.to_csv(f'Results/{output_file_name}', index=False)

In [10]:
rule

{'condition': <function __main__.<lambda>(hfsu, hfse)>, 'result': None}

In [9]:
temp_df

,Name,ExternalId,Date UTC,Time UTC,Device,Mode,Test,Direction,Position,L Reps,...,L Avg Force (N),R Avg Force (N),Avg Imbalance,L Avg Ratio,R Avg Ratio,L Max Impulse (Ns),R Max Impulse (Ns),Impulse Imbalance (%),Notes,Result
886,Subject 4,NaN,2023-11-06,19:23:00,ForceFrame-2086,Bar + Frame,Hip Flexion,Pull,Hip Flexion - Seated,4,...,437.375,486.5,10.097636,0.0,0.0,6244.195,5177.755,-17.078903,NaN,None
